# **lsstypes**: translation from external types


Let's review how to translate some measurements to **lsstypes**.

## pypower

In [1]:
from pathlib import Path
import numpy as np

from lsstypes import write, read
from lsstypes.external import from_pypower, from_pycorr

def generate_catalogs(size=100000, boxsize=(500,) * 3, offset=(1000., 0., 0), seed=42):
    rng = np.random.RandomState(seed=seed)
    positions = np.column_stack([o + rng.uniform(0., 1., size) * b for o, b in zip(offset, boxsize)])
    weights = rng.uniform(0.5, 1., size)
    return positions, weights

def generate_pypower():
    from pypower import CatalogFFTPower
    kedges = np.linspace(0., 0.2, 11)
    data_positions1, data_weights1 = generate_catalogs(seed=42)
    randoms_positions1, randoms_weights1 = generate_catalogs(seed=43)
    poles = CatalogFFTPower(data_positions1=data_positions1, data_weights1=data_weights1, randoms_positions1=randoms_positions1, randoms_weights1=randoms_weights1,
                            edges=kedges, ells=(0, 2, 4), nmesh=64, resampler='tsc', interlacing=2, los=None, position_type='pos', dtype='f8').poles
    return poles

test_dir = Path('_tests')


pypoles = generate_pypower()
poles = from_pypower(pypoles)
print(poles)
assert np.allclose(poles.value(), pypoles.power.ravel())

# I/O
fn = test_dir / 'poles.h5'
poles.write(fn)
poles = read(fn)

# Rebinning
poles = poles.select(k=slice(0, None, 2))
pypoles = pypoles[:(pypoles.shape[0] // 2) * 2:2]
assert np.allclose(poles.get(0).coords('k'), pypoles.k, equal_nan=True)
assert np.allclose(poles.value(), pypoles.power.ravel())

Mesh2SpectrumPoles(labels=[{'ells': 0}, {'ells': 2}, {'ells': 4}], size=30)


## pycorr

In [2]:
def generate_pycorr():
    from pycorr import TwoPointCorrelationFunction
    data_positions1, data_weights1 = generate_catalogs(seed=42)
    randoms_positions1, randoms_weights1 = generate_catalogs(seed=43)
    edges = (np.linspace(0., 101, 51), np.linspace(-1., 1., 101))
    return TwoPointCorrelationFunction('smu', edges, data_positions1=data_positions1, data_weights1=data_weights1,
                                        randoms_positions1=randoms_positions1, randoms_weights1=randoms_weights1,
                                        engine='corrfunc', position_type='pos', nthreads=4)

pycorr = generate_pycorr()
corr = from_pycorr(pycorr)

# I/O
fn = test_dir / 'corr.h5'
corr.write(fn)
corr = read(fn)
print(corr)

assert np.allclose(corr.coords(axis='s'), pycorr.sepavg(axis=0), equal_nan=True)
assert np.allclose(corr.value(), pycorr.corr, equal_nan=True)

# Rebinning
corr = corr.select(s=slice(0, None, 2))
pycorr = pycorr[:(pycorr.shape[0] // 2) * 2:2]
assert np.allclose(corr.coords(axis='s'), pycorr.sepavg(axis=0), equal_nan=True)
assert np.allclose(corr.value(), pycorr.corr, equal_nan=True)

# To multipoles
xi = corr.project(ells=[0, 2, 4])
assert np.allclose(xi.value(), np.ravel(pycorr(ells=[0, 2, 4])))

Count2Correlation(labels=[{'count_name': 'DD'}, {'count_name': 'RR'}, {'count_name': 'DR'}, {'count_name': 'RD'}], size=5000)
